In [32]:
from groq import Groq

client = Groq(
    api_key="",
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "si fueras un tipo de empanada, de cual serias?",
        }
    ],
    model="mixtral-8x7b-32768",
)

print(chat_completion.choices[0].message.content)

¡Interesante pregunta! Si fuera una empanada, sería una empanada de pollo y olivas. Me encanta la combinación de sabores y texturas que ofrece esta variedad de empanada. El pollo tierno y jugoso, las aceitunas con un toque salado y ligeramente amargas... ¡mmm, delicioso! Además, esta combinación es muy popular en Argentina, país al que estoy muy unido por mi pasión por el fútbol y la cultura latinoamericana.


## Usando langchain para chat completion

In [33]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="bert-base-multilingual-uncased")

# Fragmentos de texto para indexar y estructura para mapear índices a textos
text_snippets = [
    "El uso de IA generativa en la educación plantea preocupaciones sobre la integridad académica, ya que los estudiantes podrían usar estas herramientas para generar trabajos sin realmente comprender el contenido, socavando el proceso de aprendizaje.",
    "La dependencia de la IA generativa en la educación puede conducir a la pérdida de habilidades críticas de pensamiento en los estudiantes, ya que confían demasiado en la tecnología para resolver problemas y generar contenido.",
    "Existe el riesgo de que la IA generativa propague sesgos y desinformación en materiales educativos, lo cual podría moldear inadvertidamente perspectivas y entendimientos erróneos entre los estudiantes.",
    "La personalización extrema del aprendizaje mediante IA generativa podría llevar a la creación de burbujas de filtro educativas, donde los estudiantes solo se exponen a ideas y conceptos que se alinean con sus conocimientos previos o preferencias.",
    "El acceso desigual a las herramientas de IA generativa podría exacerbar la brecha educativa entre los estudiantes de diferentes entornos socioeconómicos, limitando las oportunidades para aquellos con menos recursos.",
    "La incorporación de IA generativa en el currículo educativo requiere de una revisión constante para asegurar que los contenidos generados sean precisos y estén actualizados, representando un desafío logístico y financiero para las instituciones educativas.",
    "El uso de tecnologías de IA generativa en la educación podría transformar positivamente la manera en que se diseñan los materiales didácticos, permitiendo una mayor personalización y adaptabilidad a los estilos de aprendizaje de cada estudiante.",
    "La implementación de IA generativa en los entornos educativos podría ofrecer oportunidades innovadoras para el aprendizaje interactivo y lúdico, aunque esto no aborda directamente los riesgos asociados.",
    "El desarrollo de nuevas tecnologías de IA generativa en el sector financiero ha llevado a una automatización eficiente de tareas y análisis de datos, lo cual no tiene relación directa con los riesgos en la educación.",
    "Las innovaciones en la agricultura de precisión mediante el uso de IA generativa han mejorado significativamente los rendimientos de los cultivos y la gestión de recursos, un tema completamente ajeno a los riesgos de la IA en la educación."
]

vector_store = FAISS.from_texts(text_snippets, embeddings)

No sentence-transformers model found with name bert-base-multilingual-uncased. Creating a new one with MEAN pooling.


In [41]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

chat = ChatGroq(temperature=0, groq_api_key="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", model_name="mixtral-8x7b-32768")

system = """ Eres un chatbot educativo que enseña a docentes y estudiantes sobre los riesgos y beneficios de usar inteligencia artificial generativa en el aula. Siempre contestas en español."""
human = """ Responde la pregunta del usuario basandote exclusivamente en la informacion provista en el siguiente contexto, si no hay informacion relevante en el contexto, debes indicarlo y no contestar usando tus conocimientos.
<contexto>
{context}
</contexto>

Pregunta: {question}"""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

# mixtral_chain = prompt | chat
# mixtral_chain.invoke({"text": "Hola, soy un docente. Como puedo saber si un estudiante esta usando chatGPT para hacer sus tareas?"})

In [30]:
query = "¿Cuáles son los principales riesgos asociados con el uso de la IA generativa en la educación y cómo pueden estos impactar el proceso de enseñanza-aprendizaje?"
docs = vector_store.similarity_search_with_score(query, k=3)
print(docs)

[(Document(page_content='El uso de IA generativa en la educación plantea preocupaciones sobre la integridad académica, ya que los estudiantes podrían usar estas herramientas para generar trabajos sin realmente comprender el contenido, socavando el proceso de aprendizaje.'), 14.711649), (Document(page_content='La implementación de IA generativa en los entornos educativos podría ofrecer oportunidades innovadoras para el aprendizaje interactivo y lúdico, aunque esto no aborda directamente los riesgos asociados.'), 16.89244), (Document(page_content='La dependencia de la IA generativa en la educación puede conducir a la pérdida de habilidades críticas de pensamiento en los estudiantes, ya que confían demasiado en la tecnología para resolver problemas y generar contenido.'), 16.980066)]


In [43]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(chat, prompt)
retriever = vector_store.as_retriever()

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [35]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [44]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chat
    | StrOutputParser()
)

In [45]:
rag_chain.invoke(query)

'Los principales riesgos asociados con el uso de la IA generativa en la educación incluyen preocupaciones sobre la integridad académica, la pérdida de habilidades críticas de pensamiento en los estudiantes y la dependencia excesiva de la tecnología. Estos riesgos pueden impactar negativamente el proceso de enseñanza-aprendizaje, ya que los estudiantes podrían usar estas herramientas para generar trabajos sin realmente comprender el contenido, socavando así el proceso de aprendizaje. Además, confiar demasiado en la IA para resolver problemas y generar contenido puede impedir que los estudiantes desarrollen habilidades críticas de pensamiento y resolución de problemas. Sin embargo, el contexto no proporciona información sobre posibles riesgos adicionales ni sobre cómo abordar estos desafíos de manera efectiva.'